# Convert Supervisely json to txt labels
All the labeling work was done in Supervise.ly.  Their json format needs to be converted to KITTI txt in the format:

```
kitti_formatted
  ├── images
  │   ├── IMAGE_0011.jpg
  │   ├── ......
  │   └── IMAGE_2697.jpg
  └── labels
      ├── IMAGE_0011.txt
      ├── ......
      └── IMAGE_2697.txt
```

The format is described here:  https://github.com/NVIDIA/DIGITS/tree/master/digits/extensions/data/objectDetection

We will need to: 
* Enumerate our Object Classes (Turn text in number 1..n)
* Convert the json to KITTI format
  * I think
* Convert to tfrecords (This splits train/val)
* Retrain


In [1]:
!pwd

/Users/mhale/wpilib/2020/NanoVision5607


In [2]:
import supervisely_lib as sly  # Supervisely Python SDK
import json # Add Python JSON module for pretty-printing.
import os.path
import os
import shutil

# Load the project meta-data.
# This does NOT load the images or their labeling data in memory.
project_in='../docker/data/2020team5607'
project_out=os.path.join(os.path.dirname(project_in),"Kitti")
project = sly.Project(project_in, sly.OpenMode.READ)

With an opened project, we can get some basic information: its name, location on disk, member datasets, the total number of images. We can also iterate over the datasets, and go over the images in each dataset.

In [3]:
print(project.meta)
# Print basic project metadata.
print("Project name: ", project.name)
print("Project directory: ", project.directory)
print("Total images: ", project.total_items)
print("Dataset names: ", project.datasets.keys())
print("\n")

ProjectMeta:
Object Classes
+--------------------+-----------+----------------+
|        Name        |   Shape   |     Color      |
+--------------------+-----------+----------------+
|    Blue Bottom     | Rectangle | [252, 200, 90] |
|        hole        | Rectangle | [13, 160, 158] |
|  Generator Switch  |  Polygon  | [188, 85, 16]  |
|     Blue Upper     | Rectangle | [91, 152, 251] |
| Green Color Wheel  |  Polygon  | [42, 153, 178] |
|     Red Upper      | Rectangle |  [149, 9, 89]  |
|      Red Hole      | Rectangle |  [208, 2, 27]  |
|  Red Color Wheel   |  Polygon  | [242, 121, 84] |
|     Red Lower      | Rectangle | [145, 13, 46]  |
|  Blue Color Wheel  |  Polygon  | [16, 11, 147]  |
| Yellow Color Wheel |  Polygon  | [201, 220, 40] |
|     Power_Cell     | Rectangle | [248, 231, 28] |
+--------------------+-----------+----------------+
Tags
+---------+------------+-----------------+
|   Name  | Value type | Possible values |
+---------+------------+-----------------+
| Inva

In [8]:
# What datasets and images are there and where are they on disk?
kitti_img_path   = os.path.join(project_out,"images")#,basename(img_path))
kitti_label_path = os.path.join(project_out,"labels")#,basename(img_path)+".txt")
os.makedirs(kitti_img_path, exist_ok=True)
os.makedirs(kitti_label_path, exist_ok=True)
for dataset in project:
    print("Dataset: ", dataset.name)
    dataset_name=dataset.name.replace(" ", "_")
    
    # A dataset item is a pair of an image and its annotation.
    # The annotation contains all the labeling information about
    # the image - segmentation masks, objects bounding boxes etc.
    # We will look at annotations in detail shortly.
    labels=[]
    for item_name in dataset:
        img_path = dataset.get_img_path(item_name)
        ann_path = dataset.get_ann_path(item_name)
        ann = sly.Annotation.load_json_file(ann_path, project.meta)
        if len(ann.labels):
            print(item_name)
            
            #print("  image: ", img_path)
            #print("  ann:   ", ann_path)
            #print("New Output will go:")
            
            #print("images:   ", kitti_img_path)
            #print("labels:   ", kitti_label_path)
            shutil.copy2(img_path, os.path.join(kitti_img_path,dataset_name+os.path.basename(img_path)))
            k=open( os.path.join(kitti_label_path,
                                 dataset_name+
                                 os.path.splitext(os.path.basename(img_path))[0]+".txt")
                   , "w")
            #print('\tLoaded annotation has {} labels and {} image tags.'.format(len(ann.labels), len(ann.img_tags)))
            print("Labels found, creating KITTI annotation")
            for label in ann.labels:
                bbox = label.geometry.to_bbox()
                newLabel=label.obj_class.name.replace(" ", "_")
                Kitti=' '.join([newLabel,
                     '0',
                     '0',
                     '0',
                     '{} {} {} {}'.format(bbox.top, bbox.left, bbox.bottom, bbox.right),
                     '0 0 0',
                     '0 0 0',
                     '0',
                     '0',"\n"])
                k.write(Kitti)
            k.close()
        #for tag in label.tags:
        #    print('Tag: ' + tag.get_compact_str())
    print()

Dataset:  filmingDay1Video
frame_00065.png
Labels found, creating KITTI annotation
frame_06010.png
Labels found, creating KITTI annotation
frame_02060.png
Labels found, creating KITTI annotation
frame_00105.png
Labels found, creating KITTI annotation
frame_00070.png
Labels found, creating KITTI annotation
frame_02895.png
Labels found, creating KITTI annotation
frame_04360.png
Labels found, creating KITTI annotation
frame_08995.png
Labels found, creating KITTI annotation
frame_02885.png
Labels found, creating KITTI annotation
frame_00060.png
Labels found, creating KITTI annotation
frame_00100.png
Labels found, creating KITTI annotation
frame_06000.png
Labels found, creating KITTI annotation
frame_00075.png
Labels found, creating KITTI annotation
frame_03080.png
Labels found, creating KITTI annotation
frame_00370.png
Labels found, creating KITTI annotation
frame_03335.png
Labels found, creating KITTI annotation
frame_03320.png
Labels found, creating KITTI annotation
frame_00365.png
Label

frame_03095.png
Labels found, creating KITTI annotation
frame_00775.png
Labels found, creating KITTI annotation
frame_04275.png
Labels found, creating KITTI annotation
frame_02610.png
Labels found, creating KITTI annotation
frame_00985.png
Labels found, creating KITTI annotation
frame_00950.png
Labels found, creating KITTI annotation
frame_00005.png
Labels found, creating KITTI annotation
frame_03915.png
Labels found, creating KITTI annotation
frame_03040.png
Labels found, creating KITTI annotation
frame_00575.png
Labels found, creating KITTI annotation
frame_02410.png
Labels found, creating KITTI annotation
frame_01045.png
Labels found, creating KITTI annotation
frame_03120.png
Labels found, creating KITTI annotation
frame_02570.png
Labels found, creating KITTI annotation
frame_00415.png
Labels found, creating KITTI annotation
frame_02955.png
Labels found, creating KITTI annotation
frame_00830.png
Labels found, creating KITTI annotation
frame_00165.png
Labels found, creating KITTI ann

Labels found, creating KITTI annotation
frame_00975.png
Labels found, creating KITTI annotation
frame_00785.png
Labels found, creating KITTI annotation
frame_02635.png
Labels found, creating KITTI annotation
frame_04250.png
Labels found, creating KITTI annotation
frame_00750.png
Labels found, creating KITTI annotation
frame_00585.png
Labels found, creating KITTI annotation
frame_06645.png
Labels found, creating KITTI annotation
frame_02580.png
Labels found, creating KITTI annotation
frame_00340.png
Labels found, creating KITTI annotation
frame_00630.png
Labels found, creating KITTI annotation
frame_00625.png
Labels found, creating KITTI annotation
frame_00180.png
Labels found, creating KITTI annotation
frame_00355.png
Labels found, creating KITTI annotation
frame_02595.png
Labels found, creating KITTI annotation
frame_06650.png
Labels found, creating KITTI annotation
frame_00590.png
Labels found, creating KITTI annotation
frame_02620.png
Labels found, creating KITTI annotation
frame_00

Labels found, creating KITTI annotation
DSC02126.jpg
Labels found, creating KITTI annotation
DSC02125.jpg
Labels found, creating KITTI annotation
DSC02131.jpg
Labels found, creating KITTI annotation
DSC02119.jpg
Labels found, creating KITTI annotation
DSC02135.jpg
Labels found, creating KITTI annotation
DSC02120.jpg
Labels found, creating KITTI annotation
DSC02134.jpg
Labels found, creating KITTI annotation
DSC02136.jpg
Labels found, creating KITTI annotation
DSC02137.jpg
Labels found, creating KITTI annotation

